In [2]:
from pathlib import Path

import torch
from torchmetrics.functional.text import word_error_rate
from torchmetrics.functional.text import char_error_rate

from src.models.carbune_module import CarbuneLitModule2
from src.models.components.carbune2020_net import Carbune2020NetAttempt1
from src.utils.io import load_alphabet

In [3]:
PATH = '../logs/train/multiruns/2024-03-20_13-11-14/0/tensorboard/version_0/checkpoints/epoch=9999-step=1900000.ckpt'
PATH = '../logs/train/multiruns/2024-03-27_14-06-58/0/checkpoints/epoch_epoch=029.ckpt'

BASE_PATH = Path('../logs/train/multiruns/2024-03-28_11-36-50/0')
CHECKPOINT_PATH = BASE_PATH / 'checkpoints/epoch000999.ckpt'

It looks like I don't have models saved :-D.

Next steps:

1. [x] Add model checkpointing w/ new config.
2. [x] RUn a short training just to get some model to play around with.
3. Use some of the following links to load model for inference. Links:
    - [1](https://lightning.ai/docs/pytorch/stable/common/lightning_module.html#save-hyperparameters), [2](https://lightning.ai/docs/pytorch/stable/common/trainer.html#inference-mode), [3](https://lightning.ai/docs/pytorch/stable/deploy/production_basic.html), [4](https://lightning.ai/docs/pytorch/stable/common/checkpointing_basic.html#nn-module-from-checkpoint).
    - G"pytorch lightning load model for inference"
    - !! https://lightning.ai/docs/pytorch/stable/common/lightning_module.html#inference-in-production or
    - https://lightning.ai/docs/pytorch/stable/common/lightning_module.html#save-hyperparameters
    - https://lightning.ai/forums/t/save-load-model-for-inference/542
    - https://pytorch.org/tutorials/recipes/recipes/saving_and_loading_a_general_checkpoint.html
4. Then load IAMonDB dataset and do inference. Consider using `Trainer` instead of DIY approach of doing inference. That's b/c it's easier and will not be seen in the promo video anyways.
4. Then run full training to get `checkpoint` data from there to play around.
4. Then load handwritten X++pagewise dataset and do inference on them
4. Then look for method to draw stroke in Jupyter Notebook.

*Note:* Make sure that I do correct pre-processing!

# Try to complete point 3

## Attempt 1

In [3]:
model = CarbuneLitModule2.load_from_checkpoint(PATH)

/localdisk/s1691089/venvs/carbune2020/lib/python3.10/site-packages/lightning/pytorch/utilities/parsing.py:198: Attribute 'decoder' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['decoder'])`.


RuntimeError: Error(s) in loading state_dict for CarbuneLitModule2:
	Unexpected key(s) in state_dict: "net.lstm_stack.weight_ih_l0", "net.lstm_stack.weight_hh_l0", "net.lstm_stack.bias_ih_l0", "net.lstm_stack.bias_hh_l0", "net.lstm_stack.weight_ih_l0_reverse", "net.lstm_stack.weight_hh_l0_reverse", "net.lstm_stack.bias_ih_l0_reverse", "net.lstm_stack.bias_hh_l0_reverse", "net.lstm_stack.weight_ih_l1", "net.lstm_stack.weight_hh_l1", "net.lstm_stack.bias_ih_l1", "net.lstm_stack.bias_hh_l1", "net.lstm_stack.weight_ih_l1_reverse", "net.lstm_stack.weight_hh_l1_reverse", "net.lstm_stack.bias_ih_l1_reverse", "net.lstm_stack.bias_hh_l1_reverse", "net.lstm_stack.weight_ih_l2", "net.lstm_stack.weight_hh_l2", "net.lstm_stack.bias_ih_l2", "net.lstm_stack.bias_hh_l2", "net.lstm_stack.weight_ih_l2_reverse", "net.lstm_stack.weight_hh_l2_reverse", "net.lstm_stack.bias_ih_l2_reverse", "net.lstm_stack.bias_hh_l2_reverse", "net.linear.weight", "net.linear.bias". 

TODO: Understand this problem! It has something todo w/ `nn.Module` input called `net`.

## Attempt 2

Based on [this](https://lightning.ai/docs/pytorch/stable/common/checkpointing_basic.html#save-hyperparameters).

In [4]:
checkpoint = torch.load(PATH, map_location=lambda storage, loc: storage)
# print(checkpoint["hyper_parameters"])
# {"learning_rate": the_value, "another_parameter": the_other_value}

In [5]:
list(checkpoint.keys())

['epoch',
 'global_step',
 'pytorch-lightning_version',
 'state_dict',
 'loops',
 'callbacks',
 'optimizer_states',
 'lr_schedulers',
 'hparams_name',
 'hyper_parameters',
 'datamodule_hparams_name',
 'datamodule_hyper_parameters']

In [6]:
checkpoint['hyper_parameters']

{'decoder': GreedyCTCDecoder(),
 'net': functools.partial(<class 'src.models.components.carbune2020_net.Carbune2020NetAttempt1'>, nodes_per_layer=64, number_of_layers=3, dropout=0.0),
 'optimizer': functools.partial(<class 'torch.optim.adam.Adam'>, lr=0.0001, weight_decay=0.0),
 'scheduler': None}

In [7]:
type(checkpoint['hyper_parameters']['decoder'])

src.utils.decoders.GreedyCTCDecoder

In [8]:
checkpoint['hyper_parameters']['net']

functools.partial(<class 'src.models.components.carbune2020_net.Carbune2020NetAttempt1'>, nodes_per_layer=64, number_of_layers=3, dropout=0.0)

In [9]:
checkpoint['hparams_name']

'kwargs'

In [10]:
list(checkpoint['state_dict'].keys())

['net.lstm_stack.weight_ih_l0',
 'net.lstm_stack.weight_hh_l0',
 'net.lstm_stack.bias_ih_l0',
 'net.lstm_stack.bias_hh_l0',
 'net.lstm_stack.weight_ih_l0_reverse',
 'net.lstm_stack.weight_hh_l0_reverse',
 'net.lstm_stack.bias_ih_l0_reverse',
 'net.lstm_stack.bias_hh_l0_reverse',
 'net.lstm_stack.weight_ih_l1',
 'net.lstm_stack.weight_hh_l1',
 'net.lstm_stack.bias_ih_l1',
 'net.lstm_stack.bias_hh_l1',
 'net.lstm_stack.weight_ih_l1_reverse',
 'net.lstm_stack.weight_hh_l1_reverse',
 'net.lstm_stack.bias_ih_l1_reverse',
 'net.lstm_stack.bias_hh_l1_reverse',
 'net.lstm_stack.weight_ih_l2',
 'net.lstm_stack.weight_hh_l2',
 'net.lstm_stack.bias_ih_l2',
 'net.lstm_stack.bias_hh_l2',
 'net.lstm_stack.weight_ih_l2_reverse',
 'net.lstm_stack.weight_hh_l2_reverse',
 'net.lstm_stack.bias_ih_l2_reverse',
 'net.lstm_stack.bias_hh_l2_reverse',
 'net.linear.weight',
 'net.linear.bias']

In [11]:
checkpoint = torch.load(PATH)

In [12]:
list(checkpoint.keys())

['epoch',
 'global_step',
 'pytorch-lightning_version',
 'state_dict',
 'loops',
 'callbacks',
 'optimizer_states',
 'lr_schedulers',
 'hparams_name',
 'hyper_parameters',
 'datamodule_hparams_name',
 'datamodule_hyper_parameters']

In [13]:
checkpoint['state_dict']

OrderedDict([('net.lstm_stack.weight_ih_l0',
              tensor([[ 0.1082,  0.1159, -0.0167],
                      [ 0.1280, -0.0162,  0.0384],
                      [-0.0613,  0.0728,  0.1095],
                      [-0.0787,  0.1204,  0.0364],
                      [ 0.0909,  0.0185,  0.0585],
                      [-0.0046,  0.1090,  0.0315],
                      [-0.0495,  0.0232, -0.0491],
                      [-0.0150, -0.0515,  0.0826],
                      [-0.0853, -0.0450, -0.0219],
                      [-0.0618, -0.0014, -0.1100],
                      [ 0.1260, -0.1189,  0.1098],
                      [ 0.0275, -0.0337,  0.0840],
                      [ 0.0324,  0.0886,  0.0226],
                      [-0.0400,  0.0323, -0.0354],
                      [ 0.0649,  0.1233,  0.0845],
                      [-0.0416,  0.0848,  0.0354],
                      [ 0.0753, -0.0667, -0.1126],
                      [-0.0494, -0.0995,  0.1013],
                      [ 0.0485,  0.06

## Attempt 3

Based on [this](https://lightning.ai/docs/pytorch/stable/common/checkpointing_basic.html#nn-module-from-checkpoint) and [that](https://pytorch.org/tutorials/recipes/recipes/saving_and_loading_a_general_checkpoint.html).

### Load model

In [4]:
checkpoint = torch.load(CHECKPOINT_PATH)
print(checkpoint.keys())

dict_keys(['epoch', 'global_step', 'pytorch-lightning_version', 'state_dict', 'loops', 'callbacks', 'optimizer_states', 'lr_schedulers', 'hparams_name', 'hyper_parameters', 'datamodule_hparams_name', 'datamodule_hyper_parameters'])


In [5]:
net_weights = {k.replace('net.', ''): v for k, v in checkpoint["state_dict"].items() if k.startswith("net.")}
# I remove 'net.' prefix because of later loading step

In [6]:
alphabet = load_alphabet(BASE_PATH / 'alphabet.json')

In [7]:
model = Carbune2020NetAttempt1(
    number_of_channels=3,
    nodes_per_layer=64,
    number_of_layers=3,
    dropout=0.0,
    alphabet=alphabet,
)

In [8]:
model.load_state_dict(net_weights)

<All keys matched successfully>

In [9]:
model

Carbune2020NetAttempt1(
  (log_softmax): LogSoftmax(dim=2)
  (lstm_stack): LSTM(3, 64, num_layers=3, bidirectional=True)
  (linear): Linear(in_features=128, out_features=82, bias=True)
)

### Load data

In [10]:
from src.data.online_handwriting_datamodule import IAMOnDBDataModule

In [28]:
dm = IAMOnDBDataModule(
    '../data/datasets/IAM-OnDB',
    batch_size=64,
    train_val_test_split=[0.8, 0.2, 0],
    num_workers=4,
    pin_memory=True,
    # limit=200,
    limit=3000,
    transform='carbune2020_xyn',
)
dm.setup()
dl = dm.val_dataloader()

/localdisk/s1691089/venvs/carbune2020/lib/python3.10/site-packages/torch/utils/data/dataset.py:414: UserWarning: Length of split at index 2 is 0. This might result in an empty dataset.
  warnings.warn(f"Length of split at index {i} is 0. "


In [30]:
for _ in range(3):
    sample_batched = next(iter(dl))
batch = sample_batched

### Do inference

In [32]:
model.eval()

Carbune2020NetAttempt1(
  (log_softmax): LogSoftmax(dim=2)
  (lstm_stack): LSTM(3, 64, num_layers=3, bidirectional=True)
  (linear): Linear(in_features=128, out_features=82, bias=True)
)

In [33]:
log_softmax = model(sample_batched['ink'])

In [34]:
log_softmax.shape

torch.Size([659, 64, 82])

In [35]:
decoder = checkpoint['hyper_parameters']['decoder']

In [36]:
decoder

GreedyCTCDecoder()

In [37]:
sample_batched['label']

tensor([[71, 73, 68,  ...,  0,  0,  0],
        [35,  1,  5,  ...,  0,  0,  0],
        [59, 53, 74,  ...,  0,  0,  0],
        ...,
        [55, 67, 65,  ...,  0,  0,  0],
        [70, 57, 75,  ...,  0,  0,  0],
        [53, 54, 67,  ...,  0,  0,  0]])

I would need alphabet (or alphabet_mapper) now to do proper inference.

G"pytorch lightning add data to checkpoint -> add alphabet from datamodule"

[This](https://github.com/Lightning-AI/pytorch-lightning/discussions/10172) could be the solution.

In [38]:
from src.data.tokenisers import AlphabetMapper

In [39]:
alphabet_mapper = AlphabetMapper( alphabet )

In [40]:
decoded_texts = decoder(log_softmax, alphabet_mapper)

In [41]:
batch['label_lengths']

[23,
 26,
 22,
 42,
 27,
 31,
 31,
 31,
 34,
 34,
 29,
 25,
 33,
 20,
 31,
 21,
 33,
 25,
 35,
 38,
 27,
 29,
 27,
 24,
 29,
 11,
 22,
 37,
 34,
 30,
 5,
 23,
 21,
 22,
 29,
 37,
 23,
 41,
 36,
 41,
 32,
 23,
 28,
 23,
 21,
 28,
 28,
 35,
 28,
 27,
 35,
 27,
 33,
 27,
 31,
 29,
 33,
 29,
 46,
 35,
 22,
 27,
 22,
 34]

In [42]:
# TODO: Could be pre-computed (using list0 in batch to avoid endless recomputation
labels = []
for i_batch in range(log_softmax.shape[1]):
    label_length = batch['label_lengths'][i_batch]
    label = batch['label'][i_batch, :label_length]
    label = [ alphabet_mapper.index_to_character(c) for c in label ]
    label = "".join(label)
    labels.append(label)

In [43]:
labels

['prmmlpb eb eZa jroaboba',
 'H &a jZhb vlr jfpboZ[ib- H',
 'dZsb Z ]ov Zka aolmmba',
 'iltbpq mlpfqflk lc qeb ]ZqbkZ fp qeb erjrp',
 'Arq fq tZp Z ebZsbkiv slf]b',
 'mlppf[ib+ ql pmbZh clo efjpbic-',
 'kfpba fq Zp [bfkd ZmmolufjZqbiv',
 'qebob tbob tfkbp ql loabo- Llpq',
 'Mlq pbbk pr]e Z mbocb]qiv [ZiZk]ba',
 'tbob bZqfkd fk LfqwoZgfj + ql pelt',
 'Sebpb iZpq Rfjlk tbkq qeolrde',
 'Aoltk clobpq plfi lc dlla',
 'lro Ziifbp lo Lo- Jeorpe]ebs , lo',
 'Zka fp teZq fp jbZkq',
 'tlk&q tZkq jb ebob qljloolt+ Zp',
 'RbZo]e lrq fk tfpalj-',
 'piZsbp- Seb Z]nrfpfqflk lc piZsbp',
 'bccb]qfsbiv ]lloafkZqbp Z',
 'jlabpq BeZoqboelrpb,pqobbq+ Klkalk+',
 'efp fjZdb tfqe mbqriZkq ]lk]bkqoZqflk-',
 'AvwZkqfkb qfjbp- ?ii qelrde',
 'Ofboob&p [Zo ZdZfk Zka ZdZfk-',
 'RbZiaoZrdeq Kqa ?ipl jZhb Z',
 'ql qeb sfqZi nrbpqflk lc',
 'sbuZqflk- Sefp fp qeb mol[ibj',
 "]iZvp- '0(-",
 'efp ]elf]b eb olpb Zka',
 'mZoqba ifhb lia cofbkap- Gb prddbpqba',
 'qeZq tef]e qeb Ibtp qllh rmlk qebj',
 'Efopq ql [fka qebj ql 

In [44]:
cer = char_error_rate(preds=decoded_texts, target=labels)
wer = word_error_rate(preds=decoded_texts, target=labels)

In [45]:
cer, wer

(tensor(0.8469), tensor(0.9971))

## Attempt n

Check point 3 references 1-4.